# Datawarehousing with MySQL - StartUP

based on the codes available with the book Dimensional Datawarehousing 
available here https://drive.google.com/file/d/1Sx6f1y2mXTswCllmyVgj_AsM9jVbOMnU/view?usp=sharing


# Install MySQL

In [ ]:
!apt -y install mysql-server > /dev/null
!/etc/init.d/mysql restart
!mysql -e 'show databases'



 * Stopping MySQL database server mysqld
   ...done.
 * Starting MySQL database server mysqld
su: warning: cannot change directory to /nonexistent: No such file or directory
   ...done.
+--------------------+
| Database           |
+--------------------+
| information_schema |
| mysql              |
| performance_schema |
| sys                |
+--------------------+


#Get Code & Data Files

In [ ]:
#!wget https://brainysoftware.com/source/9780975212820.zip
# archive available here https://drive.google.com/file/d/1qaiPXbxtIivCbBLmWiYvHMKAjAmNn2Y0/view?usp=sharing
#!gdown https://drive.google.com/uc?id=1qaiPXbxtIivCbBLmWiYvHMKAjAmNn2Y0
!wget -q -O dwCode.zip https://github.com/Praxis-QR/RDWH/blob/main/dwMySQL/dwCode.zip?raw=true

In [ ]:
#!unzip 9780975212820.zip
!unzip dwCode.zip

Archive:  dwCode.zip
   creating: scripts/
  inflating: scripts/across_all_dimensions.sql  
  inflating: scripts/across_date.sql  
  inflating: scripts/across_date_customer.sql  
  inflating: scripts/across_date_product.sql  
  inflating: scripts/additive_data.sql  
  inflating: scripts/add_campaign_session.sql  
  inflating: scripts/add_four_milestones.sql  
  inflating: scripts/add_sales_order_quantity.sql  
  inflating: scripts/add_two_sales_orders.sql  
  inflating: scripts/annual_aggregation.sql  
  inflating: scripts/annual_tables.sql  
  inflating: scripts/balance_across_months.sql  
  inflating: scripts/balance_across_products.sql  
  inflating: scripts/band_dim.sql    
  inflating: scripts/band_ini.sql    
  inflating: scripts/band_regular.sql  
  inflating: scripts/campaign_session.sql  
  inflating: scripts/campaign_session_20.sql  
  inflating: scripts/campaign_session_21.sql  
  inflating: scripts/campaign_session_path.sql  
  inflating: scripts/campaign_stg_20.sql  
  inf

#Part 1 - Fundamentals

## Chapter 1 Initial Setup

In [ ]:
#Create User ID
#!cat scripts/create_user_id.sql
!mysql < scripts/create_user_id.sql

ERROR 1064 (42000) at line 7: You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'IDENTIFIED BY 'pw'' at line 1


In [ ]:
#!cat scripts/create_databases.sql
!mysql -udwid -ppw < scripts/create_databases.sql

mysql: [Warning] Using a password on the command line interface can be insecure.
ERROR 1045 (28000): Access denied for user 'dwid'@'localhost' (using password: YES)


In [ ]:
#!cat scripts/create_dw_tables.sql
!mysql -udwid -ppw < scripts/create_dw_tables.sql

mysql: [Warning] Using a password on the command line interface can be insecure.
ERROR 1045 (28000): Access denied for user 'dwid'@'localhost' (using password: YES)


In [ ]:
!mysql dw -e "show tables"

ERROR 1049 (42000): Unknown database 'dw'


In [ ]:
!mysql -udwid -ppw < scripts/customer_sk.sql

mysql: [Warning] Using a password on the command line interface can be insecure.
ERROR 1045 (28000): Access denied for user 'dwid'@'localhost' (using password: YES)


In [ ]:
!mysql -udwid -ppw < scripts/more_customer_sk.sql

mysql: [Warning] Using a password on the command line interface can be insecure.


In [ ]:
!mysql dw -e "select * from customer_dim"

+-------------+-----------------+------------------+-------------------------+-------------------+---------------+----------------+----------------+-------------+
| customer_sk | customer_number | customer_name    | customer_street_address | customer_zip_code | customer_city | customer_state | effective_date | expiry_date |
+-------------+-----------------+------------------+-------------------------+-------------------+---------------+----------------+----------------+-------------+
|           1 |               1 | Big Customers    | 7500 Louise Dr.         |             17050 | Mechanicsburg | PA             | 2022-09-29     | 9999-12-31  |
|           2 |               2 | Small Stores     | 2500 Woodland St.       |             17055 | Pittsburgh    | PA             | 2022-09-29     | 9999-12-31  |
|           3 |               3 | Medium Retailers | 1111 Ritter Rd.         |             17055 | Pittsburgh    | PA             | 2022-09-29     | 9999-12-31  |
|           4 |       

In [ ]:
#since it was not possible to set the system date on Colab, the date was changed manually
!mysql dw -e "update customer_dim set effective_date = '2007-02-01' "

In [ ]:
!mysql dw -e "select * from customer_dim"

+-------------+-----------------+------------------+-------------------------+-------------------+---------------+----------------+----------------+-------------+
| customer_sk | customer_number | customer_name    | customer_street_address | customer_zip_code | customer_city | customer_state | effective_date | expiry_date |
+-------------+-----------------+------------------+-------------------------+-------------------+---------------+----------------+----------------+-------------+
|           1 |               1 | Big Customers    | 7500 Louise Dr.         |             17050 | Mechanicsburg | PA             | 2007-02-01     | 9999-12-31  |
|           2 |               2 | Small Stores     | 2500 Woodland St.       |             17055 | Pittsburgh    | PA             | 2007-02-01     | 9999-12-31  |
|           3 |               3 | Medium Retailers | 1111 Ritter Rd.         |             17055 | Pittsburgh    | PA             | 2007-02-01     | 9999-12-31  |
|           4 |       

# Chapter 2 Dimension History

In [ ]:
# creating the customer_stg table
!mysql -udwid -ppw < scripts/create_customer_stg.sql

mysql: [Warning] Using a password on the command line interface can be insecure.
ERROR 1290 (HY000) at line 26: The MySQL server is running with the --secure-file-priv option so it cannot execute this statement


In [ ]:
# Applying SCD1 to the customer names in customer_dim 
!mysql -udwid -ppw < scripts/scd1.sql

mysql: [Warning] Using a password on the command line interface can be insecure.


In [ ]:
!mysql dw -e "select * from customer_stg"

In [ ]:
# Contents of the table
!mysql dw -e "select * from customer_dim \G"

*************************** 1. row ***************************
            customer_sk: 1
        customer_number: 1
          customer_name: Big Customers
customer_street_address: 7500 Louise Dr.
      customer_zip_code: 17050
          customer_city: Mechanicsburg
         customer_state: PA
         effective_date: 2007-02-01
            expiry_date: 9999-12-31
*************************** 2. row ***************************
            customer_sk: 2
        customer_number: 2
          customer_name: Small Stores
customer_street_address: 2500 Woodland St.
      customer_zip_code: 17055
          customer_city: Pittsburgh
         customer_state: PA
         effective_date: 2007-02-01
            expiry_date: 9999-12-31
*************************** 3. row ***************************
            customer_sk: 3
        customer_number: 3
          customer_name: Medium Retailers
customer_street_address: 1111 Ritter Rd.
      customer_zip_code: 17055
          customer_city: Pittsburgh
 

In [ ]:
# Create table product_stg
!mysql -udwid -ppw < scripts/create_product_stg.sql

mysql: [Warning] Using a password on the command line interface can be insecure.


In [ ]:
# loading the data in product_stg
!mysql -udwid -ppw < scripts/load_product_stg.sql

mysql: [Warning] Using a password on the command line interface can be insecure.
ERROR 1290 (HY000) at line 17: The MySQL server is running with the --secure-file-priv option so it cannot execute this statement


In [ ]:
!mysql dw -e "select * from product_stg"

In [ ]:
!mysql dw -e "select * from product_dim"

In [ ]:
# Applying SCD2 to the product in product_dim 
!mysql -udwid -ppw < scripts/scd2.sql

mysql: [Warning] Using a password on the command line interface can be insecure.


In [ ]:
!mysql dw -e "select * from product_dim \G"

In [ ]:
!mysql dw -e "update product_dim set effective_date = '2007-02-03'"

In [ ]:
!mysql dw -e "select * from product_dim \G"

In [ ]:
# loading the more data in product_stg
!mysql -udwid -ppw < scripts/load_product_stg.sql

mysql: [Warning] Using a password on the command line interface can be insecure.
ERROR 1290 (HY000) at line 17: The MySQL server is running with the --secure-file-priv option so it cannot execute this statement


In [ ]:
!mysql dw -e "select * from product_stg"

In [ ]:
!mysql dw -e "select * from product_dim"

In [ ]:
# Applying SCD2 to the product in product_dim 
!mysql -udwid -ppw < scripts/scd2.sql

mysql: [Warning] Using a password on the command line interface can be insecure.


In [ ]:
!mysql dw -e "select * from product_dim \G"

In [ ]:
!mysql dw -e "update product_dim set effective_date = '2007-02-03'"

In [ ]:
!mysql dw -e "select * from product_dim \G"

# Chapter 3 Measure Additivity


In [ ]:
# Load data into order_dim table
!mysql -udwid -ppw < scripts/additive_data.sql

mysql: [Warning] Using a password on the command line interface can be insecure.


In [ ]:
!mysql dw -e "select * from order_dim"

+----------+--------------+----------------+-------------+
| order_sk | order_number | effective_date | expiry_date |
+----------+--------------+----------------+-------------+
|        1 |            1 | 2022-09-29     | 9999-12-31  |
|        2 |            2 | 2022-09-29     | 9999-12-31  |
|        3 |            3 | 2022-09-29     | 9999-12-31  |
|        4 |            4 | 2022-09-29     | 9999-12-31  |
|        5 |            5 | 2022-09-29     | 9999-12-31  |
|        6 |            6 | 2022-09-29     | 9999-12-31  |
|        7 |            7 | 2022-09-29     | 9999-12-31  |
|        8 |            8 | 2022-09-29     | 9999-12-31  |
|        9 |            9 | 2022-09-29     | 9999-12-31  |
|       10 |           10 | 2022-09-29     | 9999-12-31  |
+----------+--------------+----------------+-------------+


In [ ]:
!mysql dw -e "select * from date_dim"

+---------+------------+------------+-------+---------+------+----------------+-------------+
| date_sk | date       | month_name | month | quarter | year | effective_date | expiry_date |
+---------+------------+------------+-------+---------+------+----------------+-------------+
|       1 | 2005-10-31 | October    |    10 |       4 | 2005 | 2022-09-29     | 9999-12-31  |
+---------+------------+------------+-------+---------+------+----------------+-------------+


In [ ]:
!mysql dw -e "select * from sales_order_fact"

+----------+-------------+------------+---------------+--------------+
| order_sk | customer_sk | product_sk | order_date_sk | order_amount |
+----------+-------------+------------+---------------+--------------+
|        1 |           1 |          2 |             1 |      1000.00 |
|        2 |           2 |          3 |             1 |      1000.00 |
|        3 |           3 |          4 |             1 |      4000.00 |
|        4 |           4 |          2 |             1 |      4000.00 |
|        5 |           5 |          3 |             1 |      6000.00 |
|        6 |           1 |          4 |             1 |      6000.00 |
|        7 |           2 |          2 |             1 |      8000.00 |
|        8 |           3 |          3 |             1 |      8000.00 |
|        9 |           4 |          4 |             1 |     10000.00 |
|       10 |           5 |          2 |             1 |     10000.00 |
+----------+-------------+------------+---------------+--------------+


In [ ]:
# Quering across all dimensions
!mysql -udwid -ppw < scripts/across_all_dimensions.sql

mysql: [Warning] Using a password on the command line interface can be insecure.
sum_of_order_amount
58000.00


In [ ]:
# Querying across the date, product, and order 
!mysql -udwid -ppw < scripts/across_date_product.sql

mysql: [Warning] Using a password on the command line interface can be insecure.
customer_number	sum_of_order_amount
1	7000.00
2	9000.00
3	12000.00
4	14000.00
5	16000.00


In [ ]:
# Querying across the date, customer, and order 
!mysql -udwid -ppw < scripts/across_date_customer.sql

mysql: [Warning] Using a password on the command line interface can be insecure.


In [ ]:
# Querying across the date and order 
!mysql -udwid -ppw < scripts/across_date.sql

mysql: [Warning] Using a password on the command line interface can be insecure.


# Chapter 4 Dimensional Queries


In [ ]:
# Script for that adds data for testing dimensional queries 
!mysql -udwid -ppw < scripts/dimensional_query_data.sql

mysql: [Warning] Using a password on the command line interface can be insecure.


In [ ]:
# Daily Aggregation   #to insert Use dw  for database use
!mysql -udwid -ppw < scripts/daily_aggregation.sql

mysql: [Warning] Using a password on the command line interface can be insecure.
ERROR 1046 (3D000) at line 7: No database selected


In [ ]:
# Annual Aggregation #to insert Use dw  for database use
!mysql -udwid -ppw < scripts/annual_aggregation.sql

mysql: [Warning] Using a password on the command line interface can be insecure.
ERROR 1046 (3D000) at line 7: No database selected


In [ ]:
# Specific query (monthly storage product sales) 
!mysql -udwid -ppw < scripts/monthly_storage.sql

mysql: [Warning] Using a password on the command line interface can be insecure.


In [ ]:
# Specific query (quarterly sales in Mechanicsburg) 
!mysql -udwid -ppw < scripts/quarterly_mechanicsburg.sql

mysql: [Warning] Using a password on the command line interface can be insecure.
customer_city	quarter	year	SUM(order_amount)	COUNT(order_sk)
Mechanicsburg	4	2005	177000.00	10


In [ ]:
!mysql dw -e "SELECT month_name, year, product_name, SUM(order_amount), COUNT(*) FROM sales_order_fact a, product_dim b, date_dim c WHERE a.product_sk = b.product_sk AND a.order_date_sk = c.date_sk GROUP BY month_name, year, product_name HAVING SUM(order_amount) >= 75000 ORDER BY month_name, year, product_name"

In [ ]:
!mysql dw -e "select * from date_dim"

+---------+------------+------------+-------+---------+------+----------------+-------------+
| date_sk | date       | month_name | month | quarter | year | effective_date | expiry_date |
+---------+------------+------------+-------+---------+------+----------------+-------------+
|       1 | 2005-10-31 | October    |    10 |       4 | 2005 | 2022-09-29     | 9999-12-31  |
|       2 | 2005-11-01 | November   |    11 |       4 | 2005 | 2022-09-29     | 9999-12-31  |
+---------+------------+------------+-------+---------+------+----------------+-------------+


In [ ]:
# Inside-out - Monthly Product Performer # correct the month at 23rd line
!mysql -udwid -ppw < scripts/monthly_product_performer.sql

mysql: [Warning] Using a password on the command line interface can be insecure.
ERROR 1055 (42000) at line 9: Expression #1 of ORDER BY clause is not in GROUP BY clause and contains nonaggregated column 'dw.c.month' which is not functionally dependent on columns in GROUP BY clause; this is incompatible with sql_mode=only_full_group_by


In [ ]:
#  Inside-out (loyal customer) 
!mysql -udwid -ppw < scripts/loyal_customer.sql

mysql: [Warning] Using a password on the command line interface can be insecure.
customer_number	year	COUNT(*)
1	2005	4


# Part 2: Extract, Transform and Load


# Chapter 5 Source Extraction

In [ ]:
# Push CDC sales orders 
!mysql -udwid -ppw < scripts/push_sales_order.sql

mysql: [Warning] Using a password on the command line interface can be insecure.


In [ ]:
!mysql dw -e "show procedure status like 'push_sales_order' \G;"

*************************** 1. row ***************************
                  Db: source
                Name: push_sales_order
                Type: PROCEDURE
             Definer: dwid@localhost
            Modified: 2022-09-29 15:36:27
             Created: 2022-09-29 15:36:27
       Security_type: DEFINER
             Comment: 
character_set_client: utf8
collation_connection: utf8_general_ci
  Database Collation: latin1_swedish_ci


In [ ]:
# Creating the sales_order table 
!mysql -udwid -ppw < scripts/create_sales_order.sql

mysql: [Warning] Using a password on the command line interface can be insecure.


In [ ]:
# Data for testing Push mode  
!mysql -udwid -ppw < scripts/push_data.sql

mysql: [Warning] Using a password on the command line interface can be insecure.


In [ ]:
# Change database
!mysql dw -e "use source;"

In [ ]:
!mysql source -e "call push_sales_order();"

In [ ]:
# Change database
!mysql source -e "use dw;"

In [ ]:
!mysql dw -e "select * from sales_order_fact;"

+----------+-------------+------------+---------------+--------------+
| order_sk | customer_sk | product_sk | order_date_sk | order_amount |
+----------+-------------+------------+---------------+--------------+
|        1 |           1 |          2 |             1 |      1000.00 |
|        2 |           2 |          3 |             1 |      1000.00 |
|        3 |           3 |          4 |             1 |      4000.00 |
|        4 |           4 |          2 |             1 |      4000.00 |
|        5 |           5 |          3 |             1 |      6000.00 |
|        6 |           1 |          4 |             1 |      6000.00 |
|        7 |           2 |          2 |             1 |      8000.00 |
|        8 |           3 |          3 |             1 |      8000.00 |
|        9 |           4 |          4 |             1 |     10000.00 |
|       10 |           5 |          2 |             1 |     10000.00 |
|       11 |           1 |          2 |             2 |     20000.00 |
|     

# Chapter 6 Populating Date Dimension

In [ ]:
# Stored procedure to pre-populate the date dimension #'0000-00-00' invalid date format
!mysql -udwid -ppw < scripts/pre_populate_date.sql

mysql: [Warning] Using a password on the command line interface can be insecure.


In [ ]:
!mysql dw -e "show procedure status like 'pre_populate_date' \G;"

*************************** 1. row ***************************
                  Db: dw
                Name: pre_populate_date
                Type: PROCEDURE
             Definer: dwid@localhost
            Modified: 2022-09-29 15:37:07
             Created: 2022-09-29 15:37:07
       Security_type: DEFINER
             Comment: 
character_set_client: utf8
collation_connection: utf8_general_ci
  Database Collation: latin1_swedish_ci


In [ ]:
!mysql dw -e "truncate date_dim;"

In [ ]:
!mysql dw -e "SET sql_mode = 'allow_invalid_dates';"


In [ ]:
# Stored procedure to pre-populate the date dimension (Effective_date having issue with 0000-00-00 / replaced with current_date)
# !mysql -udwid -ppw < scripts/pre_populate_date.sql
!mysql dw -e "call pre_populate_date('2007-01-01', '2010-12-31');"

ERROR 1292 (22007) at line 1: Incorrect date value: '0000-00-00' for column 'effective_date' at row 1


In [ ]:
!mysql dw -e "select count(0) from date_dim;"

+----------+
| count(0) |
+----------+
|        0 |
+----------+


In [ ]:
!mysql dw -e "select * from date_dim limit 10 \G"

In [ ]:
# Daily date population   #Changed '0000-00-00' as its an incorrect date format
!mysql -udwid -ppw < scripts/daily_date.sql

mysql: [Warning] Using a password on the command line interface can be insecure.
ERROR 1292 (22007) at line 9: Incorrect date value: '0000-00-00' for column 'effective_date' at row 1


In [ ]:
!mysql dw -e "select * from date_dim  where date= '2007-02-07' \G;"

In [ ]:
# Loading dates from the source 
!mysql -udwid -ppw < scripts/source_date.sql

mysql: [Warning] Using a password on the command line interface can be insecure.
ERROR 1292 (22007) at line 9: Incorrect date value: '0000-00-00' for column 'effective_date' at row 5


In [ ]:
!mysql dw -e "select * from source.sales_order \G"

*************************** 1. row ***************************
   order_number: 17
customer_number: 1
   product_code: 1
     order_date: 2007-02-06
     entry_date: 2007-02-06
   order_amount: 1000.00
*************************** 2. row ***************************
   order_number: 18
customer_number: 2
   product_code: 1
     order_date: 2007-02-06
     entry_date: 2007-02-06
   order_amount: 1000.00
*************************** 3. row ***************************
   order_number: 19
customer_number: 3
   product_code: 1
     order_date: 2007-02-06
     entry_date: 2007-02-06
   order_amount: 4000.00
*************************** 4. row ***************************
   order_number: 20
customer_number: 4
   product_code: 1
     order_date: 2007-02-06
     entry_date: 2007-02-06
   order_amount: 4000.00


In [ ]:
!mysql dw -e "select * from date_dim WHERE date= '2007-02-06'\G"

In [ ]:
# Adding more dates from additional sales orders 
!mysql -udwid -ppw < scripts/more_sales_order.sql

mysql: [Warning] Using a password on the command line interface can be insecure.


In [ ]:
!mysql -udwid -ppw < scripts/source_date.sql   #changed to CRRENT_DATE as '0000-00-00' is incorrect

mysql: [Warning] Using a password on the command line interface can be insecure.
ERROR 1292 (22007) at line 9: Incorrect date value: '0000-00-00' for column 'effective_date' at row 5


In [ ]:
!mysql dw -e "select * from date_dim \G"

# Chapter 7 Initial Population


In [ ]:
# DW initial population 
!mysql -udwid -ppw < scripts/dw_initial.sql

mysql: [Warning] Using a password on the command line interface can be insecure.
ERROR 1290 (HY000) at line 9: The MySQL server is running with the --secure-file-priv option so it cannot execute this statement


In [ ]:
# Script for truncating the tables 
!mysql -udwid -ppw < scripts/truncate_tables.sql

mysql: [Warning] Using a password on the command line interface can be insecure.


In [ ]:
!mysql dw -e "call pre_populate_date ('2005-03-01', '2010-12-31');"

In [ ]:
!mysql -udwid -ppw < scripts/sales_order_initial.sql

mysql: [Warning] Using a password on the command line interface can be insecure.


In [ ]:
!mysql -udwid -ppw < scripts/dw_initial.sql

mysql: [Warning] Using a password on the command line interface can be insecure.


In [ ]:
!mysql -udwid -ppw < scripts/confirm_initial_population.sql

mysql: [Warning] Using a password on the command line interface can be insecure.
order_sk	customer_name	product_name	date	amount
1	Medium Retailers	Flat Panel	2005-03-01	4000.00
2	Good Companies	Regular Hard Disk Drive	2005-04-15	4000.00
3	Wonderful Shops	Floppy Drive	2005-05-20	6000.00
4	Loyal Clients	Flat Panel	2005-07-30	6000.00
5	Distinguished Partners	Regular Hard Disk Drive	2005-09-01	8000.00
6	Really Large Customers	Floppy Drive	2005-11-10	8000.00
7	Small Stores	Flat Panel	2006-01-05	1000.00
8	Medium Retailers	Regular Hard Disk Drive	2006-02-10	1000.00
9	Good Companies	Floppy Drive	2006-03-15	2000.00
10	Wonderful Shops	Flat Panel	2006-04-20	2500.00
11	Loyal Clients	Regular Hard Disk Drive	2006-05-30	3000.00
12	Distinguished Partners	Floppy Drive	2006-06-01	3500.00
13	Really Large Customers	Flat Panel	2006-07-15	4000.00
14	Small Stores	Regular Hard Disk Drive	2006-08-30	4500.00
15	Medium Retailers	Floppy Drive	2006-09-05	1000.00
16	Good Companies	Flat Panel	2006-10-05	1000.00
17	

In [ ]:
!mysql dw -e "select d.order_sk, customer_name, product_name, date, order_amount amount from sales_order_fact a, customer_dim b, product_dim c, order_dim d, date_dim e WHERE    a.customer_sk = b.customer_sk AND a.product_sk = c.product_sk AND a.order_sk = d.order_sk AND a.order_date_sk = e.date_sk"

+----------+------------------------+-------------------------+------------+---------+
| order_sk | customer_name          | product_name            | date       | amount  |
+----------+------------------------+-------------------------+------------+---------+
|        1 | Medium Retailers       | Flat Panel              | 2005-03-01 | 4000.00 |
|        2 | Good Companies         | Regular Hard Disk Drive | 2005-04-15 | 4000.00 |
|        3 | Wonderful Shops        | Floppy Drive            | 2005-05-20 | 6000.00 |
|        4 | Loyal Clients          | Flat Panel              | 2005-07-30 | 6000.00 |
|        5 | Distinguished Partners | Regular Hard Disk Drive | 2005-09-01 | 8000.00 |
|        6 | Really Large Customers | Floppy Drive            | 2005-11-10 | 8000.00 |
|        7 | Small Stores           | Flat Panel              | 2006-01-05 | 1000.00 |
|        8 | Medium Retailers       | Regular Hard Disk Drive | 2006-02-10 | 1000.00 |
|        9 | Good Companies         | Flopp

# Chapter 8 Regular Population

In [ ]:
!mysql -udwid -ppw < scripts/sales_order_regular.sql

mysql: [Warning] Using a password on the command line interface can be insecure.


In [ ]:
#  Daily dw regular population 
!mysql -udwid -ppw < scripts/dw_regular.sql 

mysql: [Warning] Using a password on the command line interface can be insecure.


In [ ]:
!mysql dw -e "select * from customer_dim \G"

*************************** 1. row ***************************
            customer_sk: 1
        customer_number: 1
          customer_name: Really Large Customers
customer_street_address: 7500 Louise Dr.
      customer_zip_code: 17050
          customer_city: Mechanicsburg
         customer_state: PA
         effective_date: 2005-03-01
            expiry_date: 9999-12-31
*************************** 2. row ***************************
            customer_sk: 2
        customer_number: 2
          customer_name: Small Stores
customer_street_address: 2500 Woodland St.
      customer_zip_code: 17055
          customer_city: Pittsburgh
         customer_state: PA
         effective_date: 2005-03-01
            expiry_date: 9999-12-31
*************************** 3. row ***************************
            customer_sk: 3
        customer_number: 3
          customer_name: Medium Retailers
customer_street_address: 1111 Ritter Rd.
      customer_zip_code: 17055
          customer_city: Pit

In [ ]:
!mysql dw -e "update product_dim set effective_date = '2007-03-01' where product_sk in (4, 5, 7, 8) "

In [ ]:
!mysql dw -e "select * from product_dim \G"

*************************** 1. row ***************************
      product_sk: 1
    product_code: 1
    product_name: Regular Hard Disk Drive
product_category: Storage
  effective_date: 2005-03-01
     expiry_date: 9999-12-31
*************************** 2. row ***************************
      product_sk: 2
    product_code: 2
    product_name: Floppy Drive
product_category: Storage
  effective_date: 2005-03-01
     expiry_date: 9999-12-31
*************************** 3. row ***************************
      product_sk: 3
    product_code: 3
    product_name: Flat Panel
product_category: Monitor
  effective_date: 2005-03-01
     expiry_date: 9999-12-31
*************************** 4. row ***************************
      product_sk: 4
    product_code: 4
    product_name: Keyboard
product_category: Peripheral
  effective_date: 2007-03-01
     expiry_date: 9999-12-31
*************************** 5. row ***************************
      product_sk: 5
    product_code: 5
    product_name:

In [ ]:
!mysql dw -e "select * from order_dim;"

+----------+--------------+----------------+-------------+
| order_sk | order_number | effective_date | expiry_date |
+----------+--------------+----------------+-------------+
|        1 |            3 | 2005-03-01     | 9999-12-31  |
|        2 |            4 | 2005-04-15     | 9999-12-31  |
|        3 |            5 | 2005-05-20     | 9999-12-31  |
|        4 |            6 | 2005-07-30     | 9999-12-31  |
|        5 |            7 | 2005-09-01     | 9999-12-31  |
|        6 |            8 | 2005-11-10     | 9999-12-31  |
|        7 |            9 | 2006-01-05     | 9999-12-31  |
|        8 |           10 | 2006-02-10     | 9999-12-31  |
|        9 |           11 | 2006-03-15     | 9999-12-31  |
|       10 |           12 | 2006-04-20     | 9999-12-31  |
|       11 |           13 | 2006-05-30     | 9999-12-31  |
|       12 |           14 | 2006-06-01     | 9999-12-31  |
|       13 |           15 | 2006-07-15     | 9999-12-31  |
|       14 |           16 | 2006-08-30     | 9999-12-31 

In [ ]:
!mysql dw -e "select * from sales_order_fact;"

+----------+-------------+------------+---------------+--------------+
| order_sk | customer_sk | product_sk | order_date_sk | order_amount |
+----------+-------------+------------+---------------+--------------+
|        1 |           3 |          3 |             1 |      4000.00 |
|        2 |           4 |          1 |            46 |      4000.00 |
|        3 |           5 |          2 |            81 |      6000.00 |
|        4 |           6 |          3 |           152 |      6000.00 |
|        5 |           7 |          1 |           185 |      8000.00 |
|        6 |           1 |          2 |           255 |      8000.00 |
|        7 |           2 |          3 |           311 |      1000.00 |
|        8 |           3 |          1 |           347 |      1000.00 |
|        9 |           4 |          2 |           380 |      2000.00 |
|       10 |           5 |          3 |           416 |      2500.00 |
|       11 |           6 |          1 |           456 |      3000.00 |
|     

# Chapter 9 Regular Population Scheduling

In [ ]:
# DW regular population batch #not possible in colab
!mysql -udwid -ppw -D dw < scripts/dw_regular.sql

mysql: [Warning] Using a password on the command line interface can be insecure.
